<a href="https://colab.research.google.com/github/ccwu0918/book-sqlfifty/blob/main/Sqlite_Flask_for_Hotel_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

## 為什麼要用資料庫？
- 可以隨心所欲的做資料的存取
- 當資料龐大時，處理效率比直接對檔案IO快上許多
- 方便跟各種服務系統串接

## 資料庫種類
- 關聯式資料庫：資料欄位須先定義清楚、穩定
- 非關聯式資料庫：資料擁有彈性結構、速度較快

## 常見的關聯式資料庫種類
- MySQL：免費、開源性質、常應用於Web，較適合中小型企業事務
- PostgreSQL：免費、開源、以商業應用導向為主
- MSSQL：Microsoft SQL Server，only on Windows，花錢就能買的完整方案。
- Oracle：公部門常用的資料庫系統，安全、效能、功能都極為齊全。
- SQLite：基於文件的資料庫，不需要架設伺服器，就能處理各種類數據。

## 常見的非關聯式資料庫種類
- MongoDB
- Couchbase
- Cassandra
- Redis
- memcache
- HBase
- Neo4J
  
![](http://image.slidesharecdn.com/mongodbrdbmsmigrationwebinar-140202060406-phpapp02/95/migrating-from-relational-databases-to-mongodb-10-638.jpg?cb=1391321199)

## Static
- 新建資料夾 `static`
- 在 `static` 資料夾中，新增檔案 `style.css`

## instance
- 新建資料夾 `instance`
- 在 `instance` 資料夾中，建立資料庫檔案 `hotel.db`

## Template
- 新建資料夾 `templates`
- 在 `templates` 資料夾中，新增檔案 `book_room.html`
- 在 `templates` 資料夾中，新增檔案 `view_bookings.html`
- 在 `templates` 資料夾中，新增檔案 `edit_booking.html`



In [ ]:
!mkdir ./static
!mkdir ./instance
!mkdir ./templates

## SQLite

``` python
import sqlite3
conn = sqlite3.connect('資料庫名稱.db') 
sql = "......"
cursor = conn.execute(sql_ins)
cursor = conn.commit() ###  會修改到資料的必須要多寫此行
conn.close()
```

### 建立資料庫/資料表
- 建立資料庫(Database/DB)：`資料庫.db`
- 建立資料表(Table)：

SQLite 型別
- INTEGER：整數
- REAL：浮點數
- TEXT：文字
- BLOB：原始資料
  
``` mysql
CREATE TABLE `資料表` (
	`欄位1`	型別,
	`欄位2`	型別,
	`.....
)
```

In [ ]:
!pip install SQLAlchemy==1.4.46

In [ ]:
# !git clone https://github.com/ccwu0918/book-sqlfifty

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db') # 如果資料庫不存在，會自動幫你建立
sql_create_table = """
CREATE TABLE IF NOT EXISTS rooms (
	`id` INTEGER PRIMARY KEY, 
	`room_number` INTEGER, 
	`room_type` TEXT, 
	`price` INTEGER, 
	`availability` TEXT, 
	`booking_date` DATE
);
"""
cursor = conn.execute(sql_create_table)
conn.close()

## CRUD

### Create 新增資料
``` mysql
INSERT INTO `資料表` (`欄位1`, `欄位2`, ...)
VALUES (值1, 值2, )
```


### 一次新增多筆資料 (INSERT INTO SELECT)


語法：

``` mysql
INSERT INTO `資料表` (`欄位1`, `欄位2`, ...)
VALUES (value1_1, value1_2, ...),
    (value2_1, value2_2, ...),
    (value3_1, value3_2, ...),
    ... ;
或利用子查詢，從其它的資料表中取得資料來作一次多筆新增：

INSERT INTO table_name (column1, column2, column3,...)
SELECT othercolumn1, othercolumn2, othercolumn3,...
FROM othertable_name;
```

In [ ]:
#@title 新增一筆訂房記錄
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')

sql_ins = """
    INSERT INTO rooms (room_number, room_type, price, availability, booking_date) 
            VALUES (101, '雙人房', 2000, 'unavailable', '2023/08/08');
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()
conn.close()

### Read 讀取資料 
``` mysql
SELECT `欄位1`, `欄位2`, ...
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```
- 顯示欄位：
    - 全選所有資料表內的欄位 `SELECT * `
    - 可自訂新欄位(運算符號和Python相同) `SELECT 欄位1, (欄位1-欄位2)*100/欄位2`
    - 可自訂順序(不需要跟資料表順序一致)
- 條件：
    - 數值比較符號大致上和Python相同，
        - 比較兩邊相等：`=`
        - 沒有整除、次方運算子(要用[函式](https://www.w3schools.com/sql/sql_ref_mysql.asp))
- 排序方式：
    - DESC 倒序(大->小)排序
    - ASC 正序(小->大)排序(可省略不寫)

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')

# SELECT `stock_id`,`date`,`open`,`close`,`high`,`low`,`volume`, `volume`*1000
# FROM `stock_date`
# WHERE `date` = '2020-03-03' or `volume` >= 40000
# ORDER BY `stock_id` DESC, `date`

sql = """
    SELECT * FROM rooms ORDER BY id, booking_date DESC;
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

### Update 更新資料
``` mysql
UPDATE `資料表`
SET `欄位1`=值1, `欄位2`=值2, ...
WHERE 條件1 and/or 條件2 ....
```

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')

id = 1
room_number = 101
room_type = "單人房"
price = 120
booking_date = "2023/07/07"

sql_upd = f"UPDATE rooms SET room_number={room_number}, room_type='{room_type}', price={price}, booking_date='{booking_date}' WHERE id={id};"

cursor = conn.execute(sql_upd)
cursor = conn.commit()
conn.close()

### Delete 刪除資料
``` mysql
DELETE 
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
```

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')

id = 1

# DELETE 
# FROM `stock_date`
# WHERE `stock_id` = 2330 and `date` = '2020-03-03'

sql_del = f"DELETE FROM rooms WHERE id={id};"

cursor = conn.execute(sql_del)
cursor = conn.commit()
conn.close()

### 處理重複資料
- 新增資料的時候先檢查資料是否存在：`SELECT`
- 設計資料庫：
    - 新增一欄位讓所有資料各擁有一唯一代碼，
        - 刪除時留下最小或最大的代碼：先用`SELECT`，再用`DELETE`
    - 建立唯一值組合：`UNIQUE INDEX`
        - 要先思考哪些欄位組合起來是唯一值

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')
sql_create_ui = """
    CREATE UNIQUE INDEX `id_date` ON `rooms`(`room_number`, `booking_date`)
"""
cursor = conn.execute(sql_create_ui)
cursor = conn.commit()
conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')
sql_ins = """
    INSERT INTO rooms (room_number, room_type, price, availability, booking_date) VALUES (101, '雙人房', 2000, 'unavailable', '2023/08/08'),
                                                    (102, '單人房', 1500, 'unavailable', '2023/09/09'),
                                                    (103, '單人房', 1500, 'unavailable', '2023/10/10'),
                                                    (104, '雙人房', 1500, 'unavailable', '2023/11/11'),                  
                                                    (105, '家庭房', 3000, 'unavailable', '2023/12/31');                      
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()

In [ ]:
# sql_ins2 = """
#     INSERT INTO rooms (room_number, room_type, price, availability, booking_date) VALUES (101, '雙人房', 2000, 'unavailable', '2023/08/08');
# """
# cursor = conn.execute(sql_ins2)
# cursor = conn.commit()
# conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('./instance/hotel.db')
sql = """
    SELECT * 
    FROM `rooms`
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

## MySQL

``` python
import pymysql
conn = pymysql.connect(host,account,passwd,dateabase)
cursor = conn.cursor() 
sql = "......"
cursor.execute(sql)
cursor.commit()
conn.close()
```

### 環境設定

### [IMDB 資料庫](https://relational.fit.cvut.cz/dataset/IMDb)
![](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg)


## INNER JOIN
- 一對多關係
    - 電影有很多演員
- 資料單一地方管理
    - `stock_list`：股票代碼、股票名稱
    - `stock_date`：股票代碼、日期、各交易原始資訊
    - 若股票名稱改名，改 `stock_list` 就好

``` mysql
SELECT `資料表n`.`欄位1`, `資料表n`.`欄位2`, ...
FROM `資料表1`
INNER JOIN `資料表2` ON 關聯條件1 and/or 條件2
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```

## 學習資源
- [w3school](https://www.w3schools.com/sql/default.asp)
- [深入淺出 SQL](https://www.tenlong.com.tw/products/9789866840166)

# Flask

## 為什麼要學架網站？
- 讓任何人不限裝置/平台/系統，只要能上網，都能很方便且易使用
- 網路爬蟲抓的就是網站，所以理解網站怎麼架設，對於增加爬蟲經驗會很有幫助

## 為什麼要用 Flask ？
- 容易上手
- 輕量化

## 環境設定

In [ ]:
!pip install flask-ngrok

In [ ]:
# !pip install flask==0.12.2

In [ ]:
!pip install pyngrok==4.1.1

## 01.註冊 ngrok 帳號 



[ngrok註冊網址](https://dashboard.ngrok.com/signup)

## 02. 建立並取得 authtoken


In [ ]:
!ngrok authtoken '29Yeppg9EErt12XNjS2DlOVpMQv_4GHpNG5xMiBCPHwm2TzmD'

In [ ]:
# !ngrok config add-authtoken 29Yeppg9EErt12XNjS2DlOVpMQv_4GHpNG5xMiBCPHwm2TzmD

## [Bootstrap](https://getbootstrap.com/)
- 包含 HTML、CSS及JavaScript的前端框架，有許多已經設定好的元素，方便開發

## [DataTables](https://datatables.net/)
- jQuery 套件，讓表格可以有更多功能操作
    - jQuery：JavaScript函式庫，簡化 HTML 與 JavaScript 之間的操作
- 此套件會把所有結果按照套件設定重新排序，亦即不會依照原先資料的排序

![](https://simpliv.files.wordpress.com/2018/04/learn-the-foundations-of-html-css-javascript-from-experts12.gif?w=640)

In [ ]:
!pip install flask_bootstrap

In [ ]:
!pip install flask_sqlalchemy



---



# 網頁樣式設定

In [ ]:
%%writefile ./static/style.css

body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
}

h1, h2 {
    text-align: center;
}

form {
    display: flex;
    flex-direction: column;
    align-items: center;
}

table {
    border-collapse: collapse;
    width: 100%;
    margin-top: 20px;
}

th, td {
    text-align: left;
    padding: 8px;
}

label {
    font-weight: bold;
}

a {
    text-decoration: none;
    color: #4CAF50;
}

th {
    background-color: #4CAF50;
    color: white;
}

tr:nth-child(even) {
    background-color: #f2f2f2;
}

input[type="text"], input[type="date"] {
    padding: 5px;
    margin-bottom: 10px;
    border-radius: 5px;
    border: none;
}

input[type="submit"] {
    background-color: #4CAF50;
    color: white;
    padding: 10px;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    align-items: center;
}

input[type="submit"]:hover {
    background-color: #3e8e41;
}

# 基礎版網頁

In [ ]:
%%writefile ./templates/book_room.html

<!DOCTYPE html>
<html>
<head>
    <title>Book Room</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>Book Room</h1>
    <h2><a href="{{ url_for('view_bookings') }}">View Bookings</a></h2>
    <form method="POST">
        <label for="room_number">Room Number:</label>
        <input type="text" id="room_number" name="room_number" required><br><br>
        <label for="room_type">Room Type:</label>
        <input type="text" id="room_type" name="room_type" required><br><br>
        <label for="price">Price:</label>
        <input type="text" id="price" name="price" required><br><br>
        <label for="booking_date">Booking Date:</label>
        <input type="date" id="booking_date" name="booking_date" required><br><br>
        <input type="submit" value="Book Room">
    </form>
</body>
</html>

In [ ]:
%%writefile ./templates/view_bookings.html

<!DOCTYPE html>
<html>
<head>
    <title>View Bookings</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>View Bookings</h1>
    <h2><a href="{{ url_for('setup') }}">Setup for Database</a></h2>
    <table>
        <thead>
            <tr>
                <th>Room Number</th>
                <th>Room Type</th>
                <th>Price</th>
                <th>Availability</th>
                <th>Booking Date</th>
                <th>Action</th>
            </tr>
        </thead>
        <tbody>
            {% for booking in bookings %}
            <tr>
                <td>{{ booking[1] }}</td>
                <td>{{ booking[2] }}</td>
                <td>{{ booking[3] }}</td>
                <td>{{ booking[4] }}</td>
                <td>{{ booking[5] }}</td>
                <td>
                    <a href="{{ url_for('edit_booking', id=booking[0]) }}">Edit</a> |
                    <a href="{{ url_for('delete_booking', id=booking[0]) }}">Delete</a>
                </td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <br>
    <form method="POST">
        <input type="submit" value="Book Room">
    </form>    

</body>
</html>

In [ ]:
%%writefile ./templates/edit_booking.html

<!DOCTYPE html>
<html>
<head>
    <title>Edit Booking</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>Edit Booking</h1>
    <h2><a href="{{ url_for('view_bookings') }}">View Bookings</a></h2>
    <form method="POST">
        {% for booking in booking %}    
        <label for="room_number">Room Number:</label>
        <input type="text" id="room_number" name="room_number" value="{{ booking[1] }}" required><br><br>
        <label for="room_type">Room Type:</label>
        <input type="text" id="room_type" name="room_type" value="{{ booking[2] }}" required><br><br>
        <label for="price">Price:</label>
        <input type="text" id="price" name="price" value="{{ booking[3] }}" required><br><br>
        <label for="booking_date">Booking Date:</label>
        <input type="date" id="booking_date" name="booking_date" value="{{ booking[5] }}" required><br><br>
        {% endfor %}
        <input type="submit" value="Update Booking">
    </form>
</body>
</html>



---



# 進階版網頁

In [ ]:
%%writefile ./templates/book_room.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">    
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}View Bookings{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">預訂房間</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_book_room">新增範例資料</a></li>                
                <li><a href="/view_bookings">訂房記錄</a></li>                
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>預訂房間</h1>
    <form method="POST">
        <label for="room_number">Room Number:</label>
        <input type="text" id="room_number" name="room_number" required><br><br>
        <label for="room_type">Room Type:</label>
        <input type="text" id="room_type" name="room_type" required><br><br>
        <label for="price">Price:</label>
        <input type="text" id="price" name="price" required><br><br>
        <label for="booking_date">Booking Date:</label>
        <input type="date" id="booking_date" name="booking_date" required><br><br>
        <input type="submit" value="確定送出">
    </form>
    </div>
</div>
{% endblock %}

In [ ]:
%%writefile ./templates/view_bookings.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">    
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}View Bookings{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">訂房記錄</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_book_room">新增範例資料</a></li>                
                <li><a href="/book_room">預訂房間</a></li>                
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>訂房記錄</h1>
    <table class="table" id="book_table">
        <thead>
            <tr>
                <th>Room Number</th>
                <th>Room Type</th>
                <th>Price</th>
                <th>Availability</th>
                <th>Booking Date</th>
                <th>Action</th>
            </tr>
        </thead>
        <tbody>
            {% for booking in bookings %}
            <tr>
                <td>{{ booking[1] }}</td>
                <td>{{ booking[2] }}</td>
                <td>{{ booking[3] }}</td>
                <td>{{ booking[4] }}</td>
                <td>{{ booking[5] }}</td>
                <td>
                    <a href="{{ url_for('edit_booking', id=booking[0]) }}">Edit</a> |
                    <a href="{{ url_for('delete_booking', id=booking[0]) }}">Delete</a>
                </td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <br>
    <form method="POST">
        <input type="submit" value="回首頁">
    </form>    
    </div>
</div>
{% endblock %}
{% block scripts %}
<script type="text/javascript">
$(document).ready(function() {
    $('#book_table').DataTable();
} );
</script>
{% endblock %}

In [ ]:
%%writefile ./templates/edit_booking.html
<head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/dataTables.bootstrap4.min.css">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">    
    <script type="text/javascript" src="https://code.jquery.com/jquery-3.3.1.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.10.20/js/dataTables.bootstrap4.min.js"></script>
</head>
{% extends "bootstrap/base.html" %}

{% block title %}View Bookings{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
            data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">導覽</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">修改訂房紀錄</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">首頁</a></li>
                <li><a href="/setup">建立資料表</a></li>
                <li><a href="/insert_book_room">新增範例資料</a></li>                
                <li><a href="/view_bookings">預訂房間</a></li>
                <li><a href="/book_room">預訂房間</a></li>                
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
    <h1>修改訂房紀錄</h1>
    <form method="POST">
        {% for booking in booking %}    
        <label for="room_number">Room Number:</label>
        <input type="text" id="room_number" name="room_number" value="{{ booking[1] }}" required><br><br>
        <label for="room_type">Room Type:</label>
        <input type="text" id="room_type" name="room_type" value="{{ booking[2] }}" required><br><br>
        <label for="price">Price:</label>
        <input type="text" id="price" name="price" value="{{ booking[3] }}" required><br><br>
        <label for="booking_date">Booking Date:</label>
        <input type="date" id="booking_date" name="booking_date" value="{{ booking[5] }}" required> ({{ booking[5] }}) <br><br>
        {% endfor %}
        <input type="submit" value="確定修改">
    </form>
    </div>
</div>
{% endblock %}



---



In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_sqlalchemy import SQLAlchemy

import sqlite3

app = Flask(__name__)

# 連接SQLite資料庫
# conn = sqlite3.connect('hotel.db')

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///hotel.db'
db = SQLAlchemy(app)
bootstrap = Bootstrap(app)
run_with_ngrok(app)


# 建立訂房系統資料表
@app.route('/setup')
def setup():
    sql = """
    CREATE TABLE IF NOT EXISTS rooms (id INTEGER PRIMARY KEY, room_number INTEGER, room_type TEXT, price INTEGER, availability TEXT, booking_date DATE)
    """
    db.engine.execute(sql)
    # return "訂房系統資料表建立成功！<a href=\"/\">回首頁</a>"
    return redirect(url_for('view_bookings')) 

@app.route('/', methods=['GET', 'POST'])

# 訂房功能
@app.route('/book_room', methods=['GET', 'POST'])
def book_room():
    if request.method == 'POST':
        room_number = request.form['room_number']
        room_type = request.form['room_type']
        price = request.form['price']
        availability = 'unavailable'
        booking_date = request.form['booking_date']

        sql = f"""
        INSERT INTO rooms (room_number, room_type, price, availability, booking_date) VALUES ({room_number}, '{room_type}', {price}, '{availability}', '{booking_date}')
        """

        db.engine.execute(sql)        
        return redirect(url_for('view_bookings'))
    return render_template('book_room.html')

# 新增多筆訂房範例記錄
@app.route('/insert_book_room')
def insert():
    sql = """
    INSERT INTO rooms (room_number, room_type, price, availability, booking_date) VALUES (101, '雙人房', 2000, 'unavailable', '2023/08/08'),
                                                    (102, '單人房', 1500, 'unavailable', '2023/09/09'),
                                                    (103, '單人房', 1500, 'unavailable', '2023/10/10'),
                                                    (104, '雙人房', 1500, 'unavailable', '2023/11/11'),                  
                                                    (105, '家庭房', 3000, 'unavailable', '2023/12/31');                      
    """
    db.engine.execute(sql)
    # return "多筆訂房範例資料新增成功！<a href=\"/\">回首頁</a>"
    return redirect(url_for('view_bookings'))

# 查看訂房紀錄
@app.route('/view_bookings', methods=['GET', 'POST'])
def view_bookings():
    if request.method == 'GET':
        sql = "SELECT * FROM rooms ORDER BY id"
        bookings = db.engine.execute(sql)
        return render_template('view_bookings.html', bookings=bookings)
    else:
        return redirect(url_for('book_room'))    

# 修改訂房紀錄
@app.route('/edit_booking/<int:id>', methods=['GET', 'POST'])
def edit_booking(id):
    if request.method == 'POST':
        room_number = request.form['room_number']
        room_type = request.form['room_type']
        price = request.form['price']
        booking_date = request.form['booking_date']
        sql = f"UPDATE rooms SET room_number={room_number}, room_type='{room_type}', price={price}, booking_date='{booking_date}' WHERE id={id};"
        db.engine.execute(sql)
        # return "資料修改成功！"
        return redirect(url_for('view_bookings'))
    else:
        sql = f"SELECT * FROM rooms WHERE id={id};"
        booking = db.engine.execute(sql)

        return render_template('edit_booking.html', booking=booking)

# 刪除訂房紀錄
@app.route('/delete_booking/<int:id>')
def delete_booking(id):
    sql = f"DELETE FROM rooms WHERE id={id};"
    db.engine.execute(sql)
    # return "資料刪除成功！"    
    return redirect(url_for('view_bookings'))

# 使用參數刪除訂房紀錄
@app.route("/delete_booking")
def delete_booking_param():
    id = request.args.get('id')
    sql = f"DELETE FROM rooms WHERE id={id};"
    db.engine.execute(sql)
    # return "資料刪除成功！"    
    return redirect(url_for('view_bookings'))   

if __name__ == '__main__':
    app.run()

#### [練習]
- 增加一個參數 booking_date
    - 如果是走 delete_booking ， url 是 `/delete_booking/001/XXX`
    - 如果是走 delete_booking_param ， url 是 `/delete_booking?id=001&booking_date=XXX`


## 學習資源
- [Flask Web 開發實戰](https://tw.pyladies.com/events/topic.html?id=35)
    - [Flask Web 開發:基於 Python 的 Web 應用開發實戰](https://www.tenlong.com.tw/products/9787115373991)
- 學習地圖
    - [前端](https://softnshare.com/webfrontendprogrammer/)
    - [後端](https://softnshare.com/backenddeveloper/)
        - VPS 主機：Linode, DigitalOcean, GCP, AWS, Azure
            - [鳥哥的 Linux 私房菜](http://linux.vbird.org/)
